# Import and Installs


In [ ]:
# colab commands for installing missing modules
!pip install mtcnn
!pip install tensorflow_addons

# Imports
import tensorflow as tf
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import os
from scipy import interpolate
from tqdm.auto import tqdm
from sklearn import metrics
from scipy.optimize import brentq
import tensorflow_addons as tfa
from tensorflow_addons.losses import metric_learning
import datetime
import mtcnn
import cv2
from google.colab import files, drive

In [2]:
# Remove colab's sample data folder
!rm -rf sample_data

##Hyperparameters

In [3]:
# Hyperparameters and constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMG_SIZE = 224                  # 224 for mobilenet, 299 for InceptionV3
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
DIR = 'CASIA-WebFace_mtcnnpy'
MARGIN = 32       
NUM_EPOCHS = 100
LFW_PAIRS_PATH = r'pairs.txt'
LFW_DIR = r'lfw_mtcnn'
TRAINLOSS = 'semiHardAngular'   # all, allAngular, semiHard, semiHardAngular
VALLOSS = 'all'                 # validation loss 
BACKBONE = 'MobileNetV3Small'   
NUM_OF_IDS = 40                 # how many different identities per batch
IMAGES_PER_PERSON = 6           # how many images per identity
CKPT_DIR = os.path.join('./checkpoints')    # Best .hdf5 model storage

# Specify which datasets should be downloaded
datasetsDownload = {'lfw':False, 'lfwCropped':True, 'casia':False, 'casiaCropped':True}

#Losses

In [4]:
@tf.function
def cosineLoss(labels, embeddings, margin, squared=False):
    # Get the distance matrix which hold the distance of each pair of images
    distances = _cosDist(embeddings)

    # Expand dimensions in preparation for triplet disntaces and assert propper shapes
    anchorToPos = tf.expand_dims(distances, 2)
    assert anchorToPos.shape[2] == 1, "Invalid shape " + str(anchorToPos.shape)
    anchorToNeg = tf.expand_dims(distances, 1)
    assert anchorToNeg.shape[1] == 1, "Invalid shape " + str(anchorToNeg.shape)

    # Compute distances of triplets: tripletLoss[i, j, k] = anchor=i, positive=j, negative=k
    tripletLoss = anchorToPos - anchorToNeg + margin

    # Create a matrix of triplet validity, valid triplets should have anchor and positive of the same ID and negative of different ID
    validityMask = _getTripletValidity(labels)
    validityMask = tf.cast(validityMask,tf.float32)

    #Multiply the validty mask with the loss, therefore retaining loss only for valid triplets
    tripletLoss = tf.multiply(validityMask, tripletLoss)

    # Easy triplets generate negative loss, remove these
    tripletLoss = tf.maximum(0., tripletLoss)

    # Calculate how many triplets still generate loss:
    lossyTriplets = tf.reduce_sum(tf.cast(tf.greater(tripletLoss, 1e-16),tf.float32))
    lossyTripletsCount = tf.reduce_sum(validityMask)
    lossyTripletsFrac = lossyTriplets / (lossyTripletsCount + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    tripletLoss = tf.reduce_sum(tripletLoss) / (lossyTriplets + 1e-16)

    return tripletLoss


# Calculates the angle between each two embeddings of the e parameter
@tf.function
def _cosDist(e):
    d1 = tf.expand_dims(e, 0)
    d1 = tf.repeat(d1, tf.shape(d1)[1], axis=0)
    d2 = tf.transpose(e)
    d2 = tf.expand_dims(e, 1)
    d2 = tf.repeat(d2, tf.shape(d2)[0], axis=1)
    distances = _angle(d1, d2)
    return distances


# Calculates the angle between two embeddings
@tf.function
def _angle(e1, e2):
    up = tf.multiply(e1, e2)
    up = tf.reduce_sum(up, axis=2)
    down = tf.tensordot(tf.norm(e1, axis=0), tf.norm(e2, axis=0),2)
    # the embeddings are already normalized, just divide by the sum of their squared values (always 2)
    res = up / 2. 
    # modify the range to <2, 0>
    return 1. - res 
   

# Returns a matrix, which hold True where the triplet is valid (eg. same positive Id, different negative Id as the anchor)
# Returns False where the IDs do not match the above condition or the triplet contains identical images
@tf.function
def _getTripletValidity(labels):
    # Check that i, j and k are distinct
    equalIndices = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    nonEqualIndices = tf.logical_not(equalIndices)
    # Create 3 matrices, each contains the informaton about equal images
    i2j = tf.expand_dims(nonEqualIndices, 2)
    i2k = tf.expand_dims(nonEqualIndices, 1)
    j2k = tf.expand_dims(nonEqualIndices, 0)

    # The valid triplets should contain different images - all images should be nonequal
    validTriplets = tf.logical_and(tf.logical_and(i2j, i2k), j2k)

    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    sameLabels = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i2j = tf.expand_dims(sameLabels, 2)
    i2k = tf.expand_dims(sameLabels, 1)

    correctClasses = tf.logical_and(i2j, tf.logical_not(i2k))

    # Combine the two conditions
    validity = tf.logical_and(validTriplets, correctClasses)

    return validity

@tf.function
def tripletLoss(labels, embeddings, margin, squared=False):
    # Get the distance matrix which hold the distance of each pair of images
    distances = metric_learning.pairwise_distance(embeddings, squared=False)

    # Expand dimensions in preparation for triplet disntaces and assert propper shapes
    anchorToPos = tf.expand_dims(distances, 2)
    assert anchorToPos.shape[2] == 1, "Invalid shape " + str(anchorToPos.shape)
    anchorToNeg = tf.expand_dims(distances, 1)
    assert anchorToNeg.shape[1] == 1, "Invalid shape " + str(anchorToNeg.shape)

    # Compute distances of triplets: tripletLoss[i, j, k] = anchor=i, positive=j, negative=k
    tripletLoss = anchorToPos - anchorToNeg + margin

    # Create a matrix of triplet validity, valid triplets should have anchor and positive of the same ID and negative of different ID
    validityMask = _getTripletValidity(labels)
    validityMask = tf.cast(validityMask,tf.float32)

    #Multiply the validty mask with the loss, therefore retaining loss only for valid triplets
    tripletLoss = tf.multiply(validityMask, tripletLoss)

    # Easy triplets generate negative loss, remove these
    tripletLoss = tf.maximum(0., tripletLoss)

    # Calculate how many triplets still generate loss:
    lossyTriplets = tf.reduce_sum(tf.cast(tf.greater(tripletLoss, 1e-16),tf.float32))
    lossyTripletsCount = tf.reduce_sum(validityMask)
    lossyTripletsFrac = lossyTriplets / (lossyTripletsCount + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    tripletLoss = tf.reduce_sum(tripletLoss) / (lossyTriplets + 1e-16)

    return tripletLoss, lossyTripletsFrac

# Strips the fraction return value
@tf.function
def tripletLossNoVal(labels, embeddings, margin, squared=False):
    loss, fraction = tripletLoss(labels, embeddings, margin, squared=False)
    return loss

Initialize losses

In [5]:
# all, allAngular, semiHard, semiHardAngular
# Training:
if TRAINLOSS == 'all': 
  trainLoss = tripletLossNoVal                                                             # Full batch, euclidean
elif TRAINLOSS == 'allAngular':
  trainLoss = cosineLoss                                                                   # Full batch, cosine
elif TRAINLOSS == 'semiHard':
  trainLoss = tfa.losses.TripletSemiHardLoss(margin = 0.2)                                 # Semi hards, euclidean
elif TRAINLOSS == 'semiHardAngular':
  trainLoss = tfa.losses.TripletSemiHardLoss(margin = 0.2, distance_metric = "angular")    # Semi hards, cosine
else:
  raiseException("Train loss hyperparameter not recognized:", TRAINLOSS)

# Validation:
if VALLOSS == 'all': 
  valLoss = tripletLoss                                                                  # Full batch, euclidean
elif VALLOSS == 'allAngular':
  valLoss = cosineLoss                                                                   # Full batch, cosine
elif VALLOSS == 'semiHard':
  valLoss = tfa.losses.TripletSemiHardLoss(margin = 0.2)                                 # Semi hards, euclidean
elif VALLOSS == 'semiHardAngular':
  valLoss = tfa.losses.TripletSemiHardLoss(margin = 0.2, distance_metric = "angular")    # Semi hards, cosine
else:
  raiseException("Validation loss hyperparameter not recognized:", VALLOSS)

#Data

In [6]:
@tf.function
def processImage(imgPath,label=None):
  # load the raw data from the file
  img = tf.io.read_file(imgPath)
  img = tf.image.decode_jpeg(img, channels=3)

  # Convert to tf float and resize to IMG_SIZE
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, (IMG_SIZE,IMG_SIZE))
  if label is not None:
    return img, label
  else:
      return img


# Set resolution to IMG_SHAPE
def setShapes(image, label=None):
  image.set_shape(IMG_SHAPE)
  if label is not None:
    label.set_shape([])
    return image, label
  else:
    return image


# Augments the 'image' with flip, jpeg quality, brightness, contranst and saturation
def augment(image, label, flipLr=True, quality=False, brightness=True, contrast=True, saturation=True):
  if flipLr:
    image = tf.image.random_flip_left_right(image)

  if quality:
    image = tf.image.random_jpeg_quality(image,80,100)

  if brightness:
    image = tf.image.random_brightness(image, 0.2)

  if contrast:
    image = tf.image.random_contrast(image, 0.2, 0.5)      

  if saturation:
    image = tf.image.random_saturation(image, 5, 10)  

  return image, label


# Creates generator containing k images from each identity in df
def permutateDS(df, k):
    uniqueLabels = np.random.permutation(df.label.unique())     # Get all labels (once/unique) and shuffle them
    for currentLabel in uniqueLabels:                                  # For each identity
        sameIdImages = df[df.label == currentLabel].sample(n=k)  # Keep rows with label == currentLabel and select k samples from it
        for i in range(k):
            image = sameIdImages.index[i]
            identity = sameIdImages.label[i]
            yield image, identity
    return


# Object holding all train and validation data, as well as the tf dataset format generated from it
# LabelsPerBatch - how many different people should be included in batch
# MinIdOccurence - how many images per person should be in batch
class TripletDataset:
    def __init__(self, labelsPerBatch, minIdOccurence):
        self.trainData      = None
        self.valData        = None
        self.minIdOccurence = minIdOccurence
        self.labelsPerBatch = labelsPerBatch
        self.trainDataset   = None
        self.valDataset     = None


    # Loads the CASIA-webface dataset (location specified by 'dir' parameter) into a dataframe
    def loadCasia(self, dir, isValidation=False):
        # Load images into pandas dataframe. Image folders are labels in this case
        images = glob(os.path.join(dir,'*/*.jpg'))            # image paths
        labels = [path.split('/')[-2] for path in images]     # folders
        data = pd.DataFrame(labels, index=images, columns=['label'])    

        # Filter indentities with fewer images than minIdOccurence
        occurences = data['label'].value_counts()
        lowCountIds = occurences[occurences < self.minIdOccurence]
        data = data[~data['label'].isin(lowCountIds.index)]
        data.label = data.label.astype('category').cat.codes

        # Save the data as either validation or train dataset
        if isValidation:
            self.valData = data
        else:
            self.trainData = data


    # Splits the dataset into train/val parts with the given ratio
    def splitDataset(self, ratio = 0.2):
        np.random.seed(42)
        valLabels       = np.random.permutation(self.trainData.label.nunique()) #select random labels for val dataset
        self.trainData  = self.trainData[self.trainData.label.isin(valLabels[int(ratio * len(valLabels)):])]
        self.valData    = self.trainData[self.trainData.label.isin(valLabels[:int(ratio * len(valLabels))])]
        

    # maps all the preprocessing functions to the dataset
    def generateDataset(self, augmentImg=True):
        # Get generators 
        self.trainGen     = partial(permutateDS, df = self.trainData, k = self.minIdOccurence)
        self.valGen       = partial(permutateDS, df = self.valData, k = self.minIdOccurence)

        # Get datasets from generator
        self.trainDataset = tf.data.Dataset.from_generator(self.trainGen, (tf.string, tf.int32))
        self.valDataset   = tf.data.Dataset.from_generator(self.valGen, (tf.string, tf.int32))
        
        # map preprocessing functions and make batches
        self.trainDataset = self.trainDataset.map(processImage, num_parallel_calls = AUTOTUNE)
        self.trainDataset = self.trainDataset.map(setShapes, num_parallel_calls = AUTOTUNE)
        self.trainDataset = self.trainDataset.batch(self.batchSize())

        # Augment 
        if augmentImg:
            self.trainDataset = self.trainDataset.map(augment, num_parallel_calls = AUTOTUNE)
        self.trainDataset = self.trainDataset.prefetch(buffer_size = AUTOTUNE)
        
        # Map preprocess for validation
        self.valDataset   = self.valDataset.map(processImage, num_parallel_calls = AUTOTUNE)
        self.valDataset   = self.valDataset.map(setShapes, num_parallel_calls = AUTOTUNE)
        self.valDataset   = self.valDataset.batch(self.batchSize())
        self.valDataset   = self.valDataset.prefetch(buffer_size = AUTOTUNE)
        

    # Returns batch size (int). 
    def batchSize(self):
        return self.minIdOccurence * self.labelsPerBatch


    # Calculates how many steps are in epoch.
    def stepsInEpoch(self):
        if self.valData is not None:
            return self.trainData.label.nunique() // self.labelsPerBatch, self.valData.label.nunique() // self.labelsPerBatch
        else:
            return self.trainData.label.nunique() // self.labelsPerBatch, 0  

Load datasets

In [ ]:
# Download the LFW dataset (unmodified)
if datasetsDownload['lfw'] and not os.path.isfile("lfw.tgz"):
  !wget http://vis-www.cs.umass.edu/lfw/lfw.tgz               
  !tar -xzf lfw.tgz
# Download the Casia dataset (unmodified)  
if datasetsDownload['casia'] and not os.path.isfile('casia.zip'):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Of_EVz-yHV7QVWQGihYfvtny9Ne8qXVz' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Of_EVz-yHV7QVWQGihYfvtny9Ne8qXVz" -O casia.zip && rm -rf /tmp/cookies.txt
  !unzip -q casia.zip
# Download the cropped LFW dataset  
if datasetsDownload['lfwCropped'] and not os.path.isfile("lfw_mtcnn.zip"):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R2wJDuZIxu1Rtx4Ei05cbokUTP0EV2YQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R2wJDuZIxu1Rtx4Ei05cbokUTP0EV2YQ" -O lfw_mtcnn.zip && rm -rf /tmp/cookies.txt
  !unzip -q lfw_mtcnn.zip
# Download the cropped CASIA dataset  
if datasetsDownload['casiaCropped'] and not os.path.isfile("CASIA-WebFace_mtcnn.zip"):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1C6zmd0eBrFZzDnenio44-W6XAlmteoW3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1C6zmd0eBrFZzDnenio44-W6XAlmteoW3" -O CASIA-WebFace_mtcnn.zip && rm -rf /tmp/cookies.txt
  !unzip -q CASIA-WebFace_mtcnn.zip 

# Download the pairs.txt file from lfw website for evaluation
if not os.path.isfile('pairs.txt'):
  !wget http://vis-www.cs.umass.edu/lfw/pairs.txt
    

#Network Architecture

In [8]:
# Constructs a NN architecture based on specified parameters:
# backbone      - specifies the CNN for feature extraction. This NN is initialized from imagenet (classfication) weights.
#               - The top layers of the backbone are excluded
# embeddingSize - Specifies the size of the output layer (and therefore the size of embedding)
# fcSize        - The size of the intermediate fully connected layer
# l2Norm        - Whether to add l2 normalization layer
def getNetwork(backbone = 'ResNet50V2', embeddingSize=128, fcSize=1024, l2Norm=True):
    # Get backbone
    if backbone == 'ResNet50V2':
      baseModel = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'ResNet101V2':
      baseModel = tf.keras.applications.ResNet101V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'ResNet152V2':
      baseModel = tf.keras.applications.ResNet152V2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet') 
    elif backbone == 'InceptionV3':
      baseModel = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')  
    elif backbone == 'MobileNetV3Large':
      baseModel = tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'MobileNetV3Small':
      baseModel = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'DenseNet169':
      baseModel = tf.keras.applications.DenseNet169(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'DenseNet121':
      baseModel = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'InceptionResNetV2':
      baseModel = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
    elif backbone == 'EfficientNetB3':
      baseModel = tf.keras.applications.EfficientNetB3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet') 
    else:
      raise Exception("Backbone network not matched to any known architecture:", backbone)

    # Set CNN to be trainable
    baseModel.trainable = True

    # Create a sequential model with backbone and new top layers
    model = tf.keras.Sequential([
        baseModel,
        tf.keras.layers.GlobalAveragePooling2D(),   # pools accross whole image, retains the filter dimension (eg. (7,7,256) -> (256))
        tf.keras.layers.Dense(fcSize, activation='relu'),   # intermediate dense layer 
        tf.keras.layers.BatchNormalization(),                 
        tf.keras.layers.Dense(embeddingSize), ])            # output layer
        
    # Add l2 normalization as lambda layer (recommended)    
    if l2Norm:
        model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))

    return model

#Align faces

In [9]:
%%script false --no-raise-error
# The cropped image should have MARGIN pixels around the face in each direction.
# If the original image does not have sufficient pixels on given side, the side of the original image
# is selected as the new margin border.

detector = mtcnn.MTCNN()

images = glob(os.path.join(CROPPEDDIR, DIR, '*/*.jpg'))

with tqdm(desc=f'Resizing', unit=' imgs', total=494414 ) as pbar:
  for imgPath in images:
    img = cv2.imread(imgPath)
    result = detector.detect_faces(img)
    if len(result) > 0:
      bounding_box = result[0]['box']
      x = np.max(bounding_box[0]-(MARGIN//2),0)
      y = np.max(bounding_box[1]-(MARGIN//2),0) 
      x2 = np.minimum(bounding_box[0]+bounding_box[2]+(MARGIN//2),img.shape[0])
      y2 = np.minimum(bounding_box[1]+bounding_box[3]+(MARGIN//2),img.shape[1])
      img = img[y:y2, x:x2]
      print("Successfull crop", imgPath)
    else:
      print('Cant crop', imgPath)
    print(os.path.join(CROPPEDDIR, imgPath))  
    print(cv2.imwrite(os.path.join(CROPPEDDIR, imgPath), img))
    pbar.update(1)  

#Training

Initialize training 

In [10]:
# Create dir for checkpoints
os.makedirs(CKPT_DIR, exist_ok=True)

# Create model and print its architecture
model = getNetwork(backbone = BACKBONE, embeddingSize=128, fcSize=1024, l2Norm=True)
model.summary()

# Create optimizer, for adam use lr between 0.0002 and 0.00005
optimizer=tf.keras.optimizers.Adam(lr=0.0001)

# Create dataset
dataset = TripletDataset(NUM_OF_IDS,IMAGES_PER_PERSON)
dataset.loadCasia(DIR)
dataset.splitDataset()                        # You can set optional split ratio, default is 80/20
dataset.generateDataset(augmentImg = False)   # Augmentation did not bring improvement

6701056/6698480 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3small (Functional (None, 7, 7, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
Total params: 2,714,864
Trainable params: 2,700,704
Non-tra

Training

In [11]:
# Single training step. Runs batch forward through NN, calculates loss and adjusts the weights.
# margin - tripletLoss margin 
@tf.function
def trainStep(X, y, model, optimizer, margin=0.2):
    with tf.GradientTape() as tape:
        # pass the images forward through the network
        embeddings = model(X, training=True)
        # calculate loss
        loss = trainLoss(y, embeddings, margin)
        # apply gradients
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss, 0

# Single validation step. You might need to modedify this if you wish to change the validation metrics
@tf.function
def valStep(X, y, model, margin=0.2):
    embeddings = model(X)
    loss, fraction = valLoss(y, embeddings, margin)
    #loss = valLoss(y, embeddings, margin)
    #fraction = 0
    return loss, fraction

In [12]:
# Initialize metrics
bestValLoss = 100
bestValValidPairs = 100
print('Batch size:', dataset.batchSize())

# For each epoch
for epoch in range(NUM_EPOCHS):
  averageEpochLoss  = tf.keras.metrics.Mean()
  epochValidPairs   = tf.keras.metrics.Mean()
  valValidPairs     = tf.keras.metrics.Mean()
  progress = 0
  trainSteps, valSteps = dataset.stepsInEpoch()

  # Train
  with tqdm(desc=f'Training - Epoch {epoch + 1}/{NUM_EPOCHS}', unit=' imgs', total= trainSteps * dataset.batchSize()) as pbar:
    for X, y in dataset.trainDataset:                       # For each batch
      loss, fraction = trainStep(X, y, model, optimizer)    
      averageEpochLoss.update_state(loss)                   # Add current batch loss
      epochValidPairs.update_state(fraction)                
      # Update progress bar
      progress += 1 / trainSteps
      pbar.set_postfix(**{'Epoch loss': averageEpochLoss.result().numpy(), 'Epoch valid pairs': epochValidPairs.result().numpy(), 'Percentage of train set': progress})
      pbar.update(dataset.batchSize())  # current batch size

  # Validate
  averageValLoss = tf.keras.metrics.Mean()
  progress = 0
  with tqdm(desc=f'Validation - Epoch {epoch + 1}/{NUM_EPOCHS}', unit=' imgs', total= valSteps*dataset.batchSize()) as pbar:
      for X, y in dataset.valDataset:                       # For each batch
          loss, fraction = valStep(X, y, model)
          averageValLoss.update_state(loss)                 # Add current batch loss
          valValidPairs.update_state(fraction)
          # Update progress bar
          progress += 1 / valSteps
          pbar.set_postfix(**{ 'Prctg of validation set': progress})
          pbar.update(dataset.batchSize())  # current batch size
  #if averageValLoss.result().numpy() < bestValLoss:
  if valValidPairs.result().numpy() < bestValValidPairs:    # Adjust based on validation metric
      print("New best result on validation dataset - Saving model")
      bestValValidPairs = valValidPairs.result().numpy()
      bestValLoss = averageValLoss.result().numpy()
      model.save_weights(os.path.join(CKPT_DIR,f"best.hdf5")) 

  print("Epoch {:03d}: Train Loss: {:.3f} || Val Loss: {:.3f}".format(epoch+1, averageEpochLoss.result(), averageValLoss.result()), end="\n\n")

Batch size: 240



New best result on validation dataset - Saving model
Epoch 001: Train Loss: 0.038 || Val Loss: 0.000



#Evaluate

In [13]:
# Download the external lfw evaluation script:
if not os.path.isdir('external'):
  os.mkdir('external')
!wget -O external/evaluate_lfw.py https://raw.githubusercontent.com/Jakub-Svoboda/DP/master/external/evaluate_lfw.py

from external.evaluate_lfw import *

--2021-04-26 13:09:39--  https://raw.githubusercontent.com/Jakub-Svoboda/DP/master/external/evaluate_lfw.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12059 (12K) [text/plain]
Saving to: ‘external/evaluate_lfw.py’

external/evaluate_l 100%[===================>]  11.78K  --.-KB/s    in 0s      

2021-04-26 13:09:40 (78.4 MB/s) - ‘external/evaluate_lfw.py’ saved [12059/12059]



In [14]:
if VALLOSS == 'all' or VALLOSS == 'semiHard':      
  testMetric = 1
else:
  testMetric = 0

In [15]:
# Create the base model from the pre-trained model MobileNet V2
model = getNetwork(backbone = BACKBONE, embeddingSize=128, fcSize=1024, l2Norm=True)
#model.load_weights('checkpoints/2021/03/02-09:56:15/epoch_49.hdf5')
model.load_weights('./checkpoints/best.hdf5')
# model.load_weights('/HOME/FaceNet/checkpoints/celeb_finetune/20200711-094426/best_epoch_19_weights.hdf5')
# Run forward pass to calculate embeddings
# Read the file containing the pairs used for testing
evaluate_LFW(model,128,use_flipped_images=False,distance_metric=testMetric,verbose=2,N_folds=10)

/content/external/evaluate_lfw.py:228: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(pairs)


Feed forward all pairs
Calculating metrics
Accuracy: 0.58267+-0.02318
Validation rate: 0.00000+-0.00000 @ FAR=0.00000
threshold : 0.00200+-0.00000
Area Under Curve (AUC): 0.589
Equal Error Rate (EER): 0.430


array([0.595     , 0.58333333, 0.58833333, 0.58833333, 0.62833333,
       0.57833333, 0.585     , 0.545     , 0.54333333, 0.59166667])

In [16]:
from google.colab import files
files.download('./checkpoints/best.hdf5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>